<a href="https://colab.research.google.com/github/Anello92/AnaliseEstatistica/blob/master/projeto_previsao_vendas_rossmann_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 Imports

In [ ]:
!pip install inflection

In [ ]:
import pandas as pd
import numpy as np
import inflection
import math
import matplotlib.pyplot as plt
import seaborn as sns

### 0.1. Helper Functions

### 0.2 Loading Data

In [ ]:
df_sales_raw = pd.read_csv('train.csv', low_memory=False)
df_store_raw = pd.read_csv('store.csv', low_memory=False)


Neste segmento, estaremos utilizando a função read_csv do pandas, que é uma poderosa biblioteca de análise e manipulação de dados em Python. A função read_csv é usada para ler dados tabulares, como um arquivo CSV, e criar um DataFrame do pandas.

O primeiro argumento para a função read_csv é o caminho para o arquivo que queremos ler. No nosso caso, o arquivo está localizado na pasta 'data' e se chama 'Train.csv'.

O segundo argumento é low_memory. Neste exemplo, estamos definindo low_memory como False, o que instrui a função read_csv a carregar todo o arquivo na memória de uma vez, em vez de ler o arquivo em pedaços. Se low_memory fosse definido como True, a função leria o arquivo em pedaços para economizar memória.

A decisão de definir low_memory como True ou False depende da capacidade de memória do seu computador. Se você tentar carregar um arquivo muito grande e seu computador tiver memória limitada, pode receber um erro ou aviso.


In [ ]:
# Merge
df_raw= pd.merge(df_sales_raw, df_store_raw, how = 'left', on = 'Store')
df_raw.sample()


Carregados os arquivos, realizamos uma operação de 'merge' (fusão) em dois conjuntos de dados utilizando a função merge do pandas. Esta operação é semelhante a um 'JOIN' em SQL, onde os dados de dois (ou mais) DataFrames são combinados com base em uma coluna (ou várias colunas) em comum.

 Para fazer isso, utilizamos a função merge do pandas, que aceita vários argumentos.

O primeiro argumento é o DataFrame que servirá como referência para a operação de 'merge'. O segundo argumento é o DataFrame que será anexado ao primeiro. O argumento how especifica o tipo de 'merge' a ser realizado. No nosso caso, o valor é 'outer', o que significa que queremos um 'merge' que inclua todas as linhas de ambos os DataFrames, independentemente de haver uma correspondência entre as colunas.

O argumento on especifica a coluna (ou colunas) que serão usadas como chave para o 'merge'. No nosso caso, a coluna é 'Store', que está presente em ambos os DataFrames.

O resultado da operação de 'merge' é armazenado em uma nova variável chamada 'df'. Em resumo, a função merge é um método da classe pandas usado para combinar dois ou mais DataFrames com base em colunas em comum.

# 1.0. Descrição dos Dados

In [ ]:
df1 = df_raw.copy()

Uma sugestão é fazer cópias do DataFrame sempre que se muda de sessão de análise em um notebook. A prática é sugerida como uma medida preventiva contra a perda de dados originais durante a manipulação dos DataFrames em sessões subsequentes.

Esse procedimento pode evitar a necessidade de reexecutar o notebook do início, economizando tempo, especialmente quando se trabalha com grandes volumes de dados.

## 1.1 Rename Columns

In [ ]:
df_raw.columns

 É recomendável renomear as colunas para nomes mais intuitivos e fáceis de lembrar. Isso pode ajudar a acelerar o desenvolvimento posterior, já que os nomes das colunas serão frequentemente usados para explorar os dados, aplicar algoritmos, fazer gráficos, entre outros.

 Embora os nomes das colunas no exemplo atual sejam bastante organizados e estejam no formato 'camel case' (alternando entre letras maiúsculas e minúsculas), isso pode não ser o caso em um ambiente real, onde os nomes das colunas podem ser muito menos intuitivos.

 Portanto, é uma boa prática revisar e, se necessário, renomear as colunas no início do processo de análise de dados.

In [ ]:
cols_old = [
'Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
'CompetitionDistance', 'CompetitionOpenSinceMonth',
'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
'Promo2SinceYear', 'PromoInterval']

Prosseguindo com a análise de dados, é uma boa prática cria uma coluna chamada 'cols_old' no DataFrame. Esta coluna será preenchida com os nomes originais das colunas do DataFrame.

Uma vez que esta lista de nomes de colunas originais foi criada, o ideal é realizar a alteração dos nomes das colunas para um formato chamado 'snake case', que consiste em todas as palavras em minúsculas separadas por sublinhados.


In [ ]:
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

Criamos a função snake_case. Para aplicá-la, utilizaremos outra função chamada map, que nos permite aplicar/mapear a função snake_case a todos os elementos de uma lista, nesse caso, a lista de nomes de colunas antigos em cols_old. O resultado dessa operação será armazenado em uma nova lista chamada 'cols_new'.

In [ ]:
df1.columns = cols_new
df1.columns

## 1.2 Data Dimensions

Um passo crucial na descrição de nossos dados é determinar as dimensões do nosso DataFrame - o número de linhas e colunas. Para isso, utilizamos o método shape que fornece essas informações.

In [ ]:
print( 'Number of Rows: {}'.format(df1.shape[0]))
print( 'Number of Cols: {}'.format(df1.shape[1]))

Ao executar essas instruções, descobrimos que nosso DataFrame possui 1.017.209 linhas distribuídas em 18 colunas. A quantidade de dados é considerável, mas gerenciável para os nossos recursos computacionais atuais. No entanto, se for necessário lidar com conjuntos de dados maiores, existem opções como servidores AWS, Google Cloud ou Kaggle, que oferecem recursos computacionais robustos gratuitamente ou com custos associados.

## 1.3 Data Types

 O próximo passo na descrição dos dados é examinar os tipos de dados presentes no DataFrame. Para isso, utilizamos o método **dtypes**, que nos permite ver a coluna e o tipo de dado correspondente sem a necessidade de usar parênteses.

In [ ]:
df1.dtypes

 Isso nos permite identificar que colunas como 'Store' são do tipo int64 e 'Date' é do tipo objeto.

No caso da coluna 'Date', queremos alterar o formato de dados para que possamos trabalhar com ele como uma data.

In [ ]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

Para fazer isso, utilizamos o método to_datetime no pandas e aplicamos diretamente na coluna 'Date' para convertê-la para o tipo de data correto. Quando verificamos novamente o tipo de dados na coluna 'Date', confirmamos que agora é datetime64, que é o formato adequado para trabalhar com datas no pandas.

### Check NA

O próximo passo na análise é a realização de uma verificação de dados nulos (NaN), que é essencial para garantir a qualidade dos dados que estamos trabalhando.

Começamos usando o método isna() em nosso DataFrame, que nos permite ver todas as linhas que contêm pelo menos uma coluna com valor ausente ou nulo. No entanto, em vez de visualizar essas linhas individualmente, queremos a soma dessas linhas por coluna, o que é feito utilizando o método sum().

In [ ]:
df1.isna().sum()

Ao executar esses comandos, podemos ver que algumas de nossas colunas não contêm valores ausentes, enquanto outras possuem alguns. Precisamos tratar esses valores ausentes e, geralmente, existem três métodos para lidar com eles:

- Descartar as linhas: Esse método envolve simplesmente descartar qualquer linha que contenha pelo menos um valor ausente. A principal vantagem é que é rápido e fácil, mas a desvantagem significativa é que você pode estar descartando informações úteis para o algoritmo aprender padrões, o que pode prejudicar a performance do seu modelo se você não tiver muitos dados.

- Usar algoritmos de aprendizado de máquina: Há algoritmos e métodos que podem preencher valores ausentes com base no comportamento geral da coluna. Por exemplo, você pode preencher os valores ausentes com a média, mediana ou moda da coluna, ou até mesmo usar algoritmos mais avançados que estimam o que o valor poderia ser se não estivesse ausente.

- Usar conhecimento de negócio: Às vezes, os valores ausentes estão lá devido a alguma lógica de negócio definida inicialmente. Se soubermos essa regra, podemos preencher os valores ausentes de forma adequada e recuperar os dados.

## 1.5 Fillout NA
Vamos iniciar uma nova subseção, denominada 'Fill-na', com o objetivo de preencher os dados ausentes.


Começaremos lidando com a coluna 'competition_distance'. Retornando à descrição da coluna, sabemos que essa coluna indica a distância, em metros, do concorrente mais próximo.

In [ ]:
# competition_distance
# competition_open_since_month
# competition_open_since_year
# promo2_since_week
# promo2_since_year
# promo_interval

### Existem três maneiras principais de lidar com dados ausentes (NaN):

A primeira maneira é excluir todos os dados ausentes. A principal vantagem desse método é a rapidez, mas a desvantagem é a perda substancial de informações.

A segunda maneira é usar algoritmos de aprendizado de máquina para substituir dados ausentes com base no comportamento da coluna. Por exemplo, podemos calcular a mediana ou a média e substituir todos os dados ausentes por esses valores. Algoritmos mais sofisticados podem fazer agrupamentos ou até mesmo prever esses valores ausentes.

A terceira maneira, e a que vamos adotar neste tutorial, é pensar do ponto de vista do negócio. Embora não sejamos especialistas, é útil fazer esse tipo de exercício. Pensar sobre o motivo pelo qual esses dados estão ausentes pode oferecer insights valiosos.



### 1. Competition Distance
Vamos considerar a coluna 'competition_distance'. Uma maneira de interpretar a ausência de dados aqui é que a distância até o concorrente mais próximo é tão grande que é efetivamente o mesmo que não haver concorrente próximo.

In [ ]:
df1['competition_distance'].max()

Com a ausência de valores em 'competition_distance', iremos substituí-los por um número extremamente grande, que chamaremos de 'max_value'. Para definir este 'max_value', vamos primeiro procurar o valor máximo existente na coluna 'competition_distance'. No nosso caso, encontramos o valor de 75.860 metros como o mais distante concorrente.

In [ ]:
df1['competition_distance'] = df1['competition_distance'].apply(
    lambda x: 200000.0 if math.isnan(x) else x)

Vamos escolher 'max_value' como 200.000 metros, um valor bem maior que a distância máxima existente na coluna 'competition_distance'. Valores ausentes serão substituídos por 'max_value', indicando que não há concorrente próximo nesses casos.

Se um valor estiver presente, será retornado como está. Aplicaremos essa lógica ao nosso conjunto de dados usando a função lambda, que será aplicada apenas na coluna 'competition_distance'. O resultado substituirá a coluna original.

Após aplicar a função e recalcular a soma de linhas NaN, vemos que não há mais valores ausentes na coluna 'competition_distance', mostrando que lidamos com os dados ausentes com sucesso.

In [ ]:
df1.isna().sum()

Agora, a coluna 'competition_distance' não possui mais valores ausentes, e o valor máximo é 200.000 metros, como definido anteriormente.

In [ ]:
df1['competition_distance'].max()

### 2. Competition Open Since Month
Avançando, vamos analisar a coluna 'competition_open_since'. Esta coluna indica o mês e o ano aproximado em que o concorrente mais próximo foi inaugurado.

Podemos supor que os valores ausentes nesta coluna podem ser por duas razões: primeiro, a loja pode não ter um concorrente próximo, logo não existe uma data de abertura para tal concorrente. Segundo, a loja pode ter um concorrente próximo, mas desconhecemos a data de abertura deste, seja porque o concorrente abriu antes da loja ou porque abriu posteriormente.

Para substituir os valores ausentes nesta coluna, copiaremos a data de venda correspondente na linha em questão para a coluna 'competition_open_since'.

In [ ]:
df1.sample()

Agora vamos considerar a coluna 'competition_open_since' em uma linha de exemplo onde uma venda foi feita. Por exemplo, se a linha representar uma venda feita pela loja 283 em 20/05/2014, onde vendeu 7300 itens, e os dados mostram um valor ausente para 'competition_open_since', substituiremos esse valor ausente com a data da venda.

Vamos extrair apenas o mês da data da venda para preencher os valores ausentes na coluna 'competition_open_since_month'. Isso se deve à nossa previsão de que variáveis temporais podem ser cruciais na modelagem do comportamento dos dados.

A lógica é que uma loja sem concorrência tem um nível estável de vendas, que provavelmente cairá quando um concorrente abrir. Com o tempo, as vendas podem se recuperar à medida que os clientes se acostumam com ambas as lojas. Assim, o tempo desde a abertura de um concorrente pode influenciar as flutuações de vendas.

Reconhecemos uma incongruência em usar a data da venda como a data de abertura do concorrente, especialmente se supomos que o concorrente ainda não abriu. No entanto, prosseguiremos com essa abordagem para avaliar seu impacto no modelo. Se necessário, podemos ajustá-la em futuras iterações.

A forma como implementaremos essa substituição será semelhante ao que fizemos para 'competition_distance'.






In [ ]:
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month
                                                if np.isnan(x['competition_open_since_month'])
                                                else x['competition_open_since_month'], axis=1)

Portanto, se a coluna 'competition_open_since_month' estiver vazia, iremos preencher com o mês da coluna 'date'.

Se essa condição não for verdadeira, iremos simplesmente retornar o valor original, pois já temos o mês em que a competição foi aberta.

Para aplicar essa lógica, vamos usar a função lambda novamente. Dentro desta função, podemos substituir 'df1' por 'x', já que tudo dentro da função é referido como 'x'.

Para aplicar isso aos nossos dados, vamos usar a função 'apply' e aplicá-la ao longo das colunas. Fazemos isso porque estamos trabalhando com mais de uma coluna - 'competition_open_since_month' e 'date'. Quando trabalhamos com mais de uma coluna, precisamos explicitamente aplicar a função ao longo das colunas.

Finalmente, o resultado dessa função será usado para substituir a coluna original 'competition_open_since_month'.






### 3. Competition Open Since Year
Vamos substituir todas as ocorrências de 'month' por 'year' na linha atual.


In [ ]:
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year
                                               if np.isnan(x['competition_open_since_year'])
                                               else x['competition_open_since_year'], axis=1)

### 4. Promo 2 Since Week
A coluna 'promo2' indica se uma loja está participando de uma continuação de uma promoção. Se o valor é zero, a loja não está participando; se é um, a loja está participando. Se os dados para 'promo2' são NaN, isso indica que a loja optou por não participar dessa promoção contínua, logo não há uma semana de início para essa promoção.

Para lidar com esses valores ausentes, usaremos uma abordagem semelhante à que usamos para a coluna 'competition_distance': substituir os NaNs pelo valor da data naquela linha. Quando calculamos a distância em tempo, isso permitirá ao algoritmo reconhecer que temos essa promoção ativa desde uma determinada semana.

Para implementar essa substituição, copiamos e colamos o código que usamos para 'competition_distance', fazendo algumas pequenas alterações. Em vez de 'competition', procuramos 'promo2', e em vez de extrair o mês da data, queremos extrair a semana.

In [ ]:
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week
                                     if np.isnan(x['promo2_since_week'])
                                     else x['promo2_since_week'], axis=1)

### 5. Promo 2 Since Year
Para implementar essa substituição, copiamos e colamos o código que usamos para 'promo2_since_week', fazendo algumas pequenas alterações para year e extrair o ano.

In [ ]:
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year
                                     if np.isnan(x['promo2_since_year'])
                                     else x['promo2_since_year'], axis=1)

Este código faz o seguinte:

- Usa a função apply para aplicar uma função lambda a cada linha do dataframe (df1.apply(lambda x: ..., axis=1)).
- Na função lambda, verificamos se 'promo2_since_year' é NaN para essa linha (if math.isnan(x['promo2_since_year'])).
- Se for NaN, substituímos pelo número da semana da data dessa linha (x['date'].year).
- Se não for NaN, mantemos o valor original (else x['promo2_since_year']).

### 6. Promo Interval
Promo Interval" descreve os meses em que uma promoção chamada "Promo 2" foi iniciada. Por exemplo, se os valores forem "Fevereiro, Maio, Agosto, Novembro", isso indica que a promoção foi realizada nesses meses específicos.

In [ ]:
month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dez'}

df1['promo_interval'].fillna(0, inplace = True)

df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0
                                                             if x['promo_interval'] == 0
                                                             else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis = 1)

In [ ]:
df1.sample(5).T

A primeira etapa na manipulação desses dados é fazer um "split" nesta coluna para criar uma lista dos meses da promoção. Em seguida, uma nova coluna chamada "month_map" será criada com base na data da promoção. Se a data da promoção estiver na lista de meses, um valor indicando que a "Promo 2" estava ativa será inserido nessa nova coluna.

Depois de criar a coluna auxiliar, os meses são convertidos de números para suas representações de string correspondentes. Por exemplo, janeiro seria representado como "1". Uma função é então aplicada para fazer a substituição direta na coluna.

Em seguida, os valores da coluna "Promo Interval" são substituídos por seus equivalentes numéricos. Para fazer isso, um dicionário é criado e aplicado à coluna. Isso cria uma coluna chamada "month_map" que contém o mês em que a promoção ocorreu.

Uma função é então aplicada para comparar se o mês de uma promoção está na lista de meses de "Promo 2". Se o mês estiver na lista, é retornado um valor de "1". Se o mês não estiver na lista, é retornado um valor de "0". Além disso, se uma loja não participou da "Promo 2" (indicado por um valor de "0" na coluna "Promo Interval"), um valor de "0" também é retornado.


In [ ]:
df1.isna().sum()

## 1.6 Change Types
Quando operações são realizadas em colunas de um conjunto de dados, é vital verificar novamente os tipos de dados dessas colunas. A razão para isso é que as operações podem inadvertidamente alterar os tipos de dados originais.

Pode haver situações em que é necessário mudar o tipo de dados de uma coluna para um diferente. Por exemplo, se quisermos converter o tipo de dados 'float' da coluna 'competition' para 'int' (um número inteiro), podemos usar o método astype().


In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)

df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

## 1.7 Descriptive Statistics
A estatística descritiva é uma ferramenta essencial para a análise de dados, sendo particularmente útil em duas áreas: obtenção de conhecimento de negócios e detecção de erros nos dados.

Em primeiro lugar, a estatística descritiva proporciona uma visão geral dos dados, permitindo obter insights sobre os negócios. Isto é feito através de métricas de estatísticas descritivas, que se dividem em duas categorias principais: **métricas de tendência central** e **métricas de dispersão**.

As métricas de tendência central, como a média e a mediana, fornecem um resumo dos dados em um único valor. Em essência, elas permitem representar uma variedade de dados através de uma média ou mediana. No entanto, embora essas métricas possam fornecer uma visão geral, elas não revelam como os dados estão distribuídos em torno dessa referência central. É aqui que entram as métricas de dispersão.

As métricas de dispersão, que incluem variância, desvio padrão e valores mínimos e máximos, fornecem informações sobre a distribuição dos dados. Elas indicam se os dados estão concentrados próximo à média ou dispersos em torno dela. Além disso, duas outras medidas de dispersão, chamadas "skewness" (assimetria) e "kurtosis" (curtose), podem ser usadas para entender melhor a distribuição dos dados. A skewness refere-se ao grau de distorção da distribuição em relação a uma distribuição normal, enquanto a curtose descreve a concentração dos dados, indicando se os dados têm um "pico" alto (curtose positiva) ou são mais dispersos (curtose negativa).

Ao trabalhar com a estatística descritiva, é importante separar as variáveis numéricas das categóricas, pois a forma como cada tipo é tratado pode variar. Para selecionar variáveis numéricas, você pode utilizar um método de seleção em sua ferramenta de análise de dados, como select_dtypes() no pandas, uma biblioteca do Python, e passar uma lista dos tipos de dados que você deseja selecionar.

In [ ]:
num_attributes = df1.select_dtypes(include = ['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude = ['int64', 'float64', 'datetime64[ns]'])

Neste exemplo, 'df1' é o DataFrame original e 'num_attributes' é o novo DataFrame contendo apenas as colunas numéricas. Podemos visualizar algumas linhas aleatórias do DataFrame usando o método sample(), como mostrado abaixo

In [ ]:
num_attributes.sample(2)

Agora, 'cat_attributes' contém todas as colunas categóricas do DataFrame original. Como antes, podemos visualizar algumas linhas aleatórias do DataFrame usando o método sample().

In [ ]:
cat_attributes.sample(2)

Após separar os dados numéricos e categóricos, podemos proceder para calcular diferentes métricas relevantes para cada tipo de dado. Por exemplo, para dados numéricos, podemos calcular a média, mediana, desvio padrão, etc. Para dados categóricos, podemos contar a frequência de diferentes categorias, encontrar a categoria mais comum, etc.

### 1.7.1 Numerical Attributes
Vamos abordar o cálculo de métricas de tendência central para variáveis numéricas. As métricas de tendência central resumem o conjunto de dados em um único valor representativo, fornecendo um 'centro' em torno do qual os dados estão distribuídos. As métricas de tendência central mais comuns são a média e a mediana.

In [ ]:
# Central Tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - std, min, max, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

Depois de todas essas métricas calculadas, podemos concatená-las em um único DataFrame para facilitar a visualização. Usamos o método concat() da biblioteca pandas para isso:

In [ ]:
# Concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

Neste ponto, 'm' é um DataFrame que contém todas as métricas calculadas para cada coluna numérica. Cada linha representa uma métrica, e cada coluna representa uma coluna numérica do DataFrame original.

Considere uma coluna 'Sales' em um conjunto de dados. Com base nas estatísticas, o valor mínimo é zero, indicando que houve dias em que nenhuma venda foi realizada, talvez devido ao fechamento de lojas. O valor máximo é de 41.000, indicando o maior volume de vendas em um dia. O 'range', ou a diferença entre o máximo e o mínimo, é, portanto, 41.000.

Quando a média e a mediana de um conjunto de dados são semelhantes, como neste caso, **indica que a distribuição é próxima de uma distribuição normal**, sem um deslocamento significativo para a esquerda ou para a direita. Além disso, o skewness (medida da assimetria da distribuição) e kurtosis (medida da "cauda" ou outliers da distribuição) também estão próximos de zero, corroborando a suposição da normalidade.


### 1.7.2 Categorical Attributes
Para realizar uma análise descritiva dos dados categóricos, é útil usar gráficos para visualizar as informações. Um tipo de gráfico muito útil para esse propósito é o gráfico de boxplot. Este gráfico permite visualizar diversas medidas estatísticas, como medidas de tendência central e dispersão, em um único lugar, facilitando a comparação entre as categorias.

Entretanto, antes de construir o boxplot, pode ser útil verificar a quantidade de níveis ou valores únicos que cada variável categórica possui. Isso pode ser feito com o uso da função nunique do pandas em Python, que retorna o número de elementos distintos em cada coluna de um dataframe. A função nunique pode ser aplicada a todas as colunas de um dataframe utilizando o método apply.



In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

No exemplo fornecido, as variáveis categóricas analisadas incluem 'state_holiday', 'store_type', 'assortment, 'promot_interval' e 'month_map'. Cada uma dessas variáveis possui três níveis distintos.

In [ ]:
sns.boxplot(x = 'state_holiday', y = 'sales', data = df1)

Estamos analisando a dispersão de 'sales' em cada nível de uma variável categórica. Para isso, podemos definir 'state_holiday' como o eixo x (a categoria) e 'sales' como o eixo y (o valor a ser medido). Os dados serão extraídos do nosso DataFrame, digamos df1, que contém os dados categóricos.

No entanto, pode ser que, ao visualizar o boxplot, os dados estejam distorcidos ou difíceis de interpretar. Uma razão comum para isso é a presença de valores que não são úteis para a análise, como vendas iguais a zero correspondendo a dias em que as lojas estão fechadas.

Para resolver esse problema, podemos filtrar os dados antes de plotar o gráfico. Por exemplo, podemos querer visualizar apenas as vendas que são maiores que zero e que ocorreram em dias que não são feriados. Para fazer isso, podemos utilizar a operação de indexação booleana do pandas para criar um novo DataFrame que satisfaz essas condições:


In [ ]:
aux1 = df1[(df1['state_holiday'] != '0') & (df1['sales'] > 0)]

plt.figure(figsize=(15, 6))  # Aqui, 15 representa a largura e 6 a altura.

plt.subplot(1, 3, 1)
sns.boxplot(x = 'state_holiday', y = 'sales', data = aux1)

plt.subplot(1, 3, 2)
sns.boxplot(x = 'store_type', y = 'sales', data = aux1)

plt.subplot(1, 3, 3)
sns.boxplot(x = 'assortment', y = 'sales', data = aux1)

plt.tight_layout()
plt.show()

Acima utilizamos a biblioteca Seaborn para criar boxplots de diferentes variáveis categóricas em relação às vendas. Por exemplo, analisamos as vendas de acordo com o tipo de feriado 'state_holiday' e o tipo de loja 'store_type'.

Para comparar visualmente todas as variáveis de uma só vez, usamos a função plt.subplot(), que nos permite traçar múltiplos gráficos lado a lado. No caso, configuramos para uma linha e três colunas, correspondendo aos três boxplots.

A mediana, indicada pela linha no meio da 'caixa' do boxplot, representa o valor do 50º percentil, ou seja, o ponto onde metade dos dados está abaixo e metade está acima. O primeiro quartil (Q1) é o 25º percentil, indicando onde 25% dos dados estão abaixo e 75% estão acima, enquanto o terceiro quartil (Q3) é o 75º percentil, marcando onde 75% dos dados estão abaixo e 25% estão acima.

Observamos que o 'state_holiday' do tipo 'B' tem uma mediana maior de vendas do que os outros tipos. Em relação aos tipos de loja em 'store_type', a loja do tipo 'B' tem mais outliers e uma mediana de vendas significativamente mais alta em comparação às outras.

Cada boxplot fornece uma rica informação sobre a distribuição das vendas para cada categoria, possibilitando identificar onde a maioria das vendas está concentrada (em torno da mediana) e como elas estão distribuídas entre o mínimo e o máximo.

Essa análise usando boxplots facilita a compreensão da relação entre diferentes variáveis categóricas e as vendas, sendo uma ferramenta poderosa para a análise exploratória de dados.